## Run inference on the Llama 2 endpoint you have created

### Query endpoint that you have created
***
The following cell provides a helper function that will be used to query your endpoint with boto3.
***

In [2]:
import json
import boto3


endpoint_name = "jumpstart-dft-meta-textgeneration-llama-2-13b-f"

def query_endpoint(payload):
    client = boto3.client("sagemaker-runtime")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps(payload),
    )
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response

### Supported Parameters

***
This model supports many parameters while performing inference. They include:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches `max_new_tokens`. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.
* **stop**: If specified, it must be a list of strings. Text generation stops if any one of the specified strings is generated.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments.

**NOTE**: If `max_new_tokens` is not defined, the model may generate up to the maximum total tokens allowed, which is 4K for these models. This may result in endpoint query timeout errors, so it is recommended to set `max_new_tokens` when possible. For 7B, 13B, and 70B models, we recommend to set `max_new_tokens` no greater than 1500, 1000, and 500 respectively, while keeping the total number of tokens less than 4K.

**NOTE**: This model only supports 'system', 'user' and 'assistant' roles, starting with 'system', then 'user' and alternating (u/a/u/a/u...).

***

### Example prompts
***
The examples in this section demonstrate how to perform text generation with conversational dialog as prompt inputs. First, define a helper function to appropriately format conversational dialog for Llama-2 chat models.
***

In [3]:
from typing import Dict, List


def format_messages(messages: List[Dict[str, str]]) -> List[str]:
    """Format messages for Llama-2 chat models.
    
    The model only supports 'system', 'user' and 'assistant' roles, starting with 'system', then 'user' and 
    alternating (u/a/u/a/u...). The last message must be from 'user'.
    """
    prompt: List[str] = []

    if messages[0]["role"] == "system":
        content = "".join(["<<SYS>>\n", messages[0]["content"], "\n<</SYS>>\n\n", messages[1]["content"]])
        messages = [{"role": messages[1]["role"], "content": content}] + messages[2:]

    for user, answer in zip(messages[::2], messages[1::2]):
        prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])

    prompt.extend(["<s>", "[INST] ", (messages[-1]["content"]).strip(), " [/INST] "])

    return "".join(prompt)


def print_messages(prompt: str, response: str) -> None:
    bold, unbold = '\033[1m', '\033[0m'
    print(f"{bold}> Input{unbold}\n{prompt}\n\n{bold}> Output{unbold}\n{response[0]['generated_text']}\n")
    
def get_response(prompt: str, response: str) -> None:
    bold, unbold = '\033[1m', '\033[0m'
    print(f"> Output{unbold}\n{response[0]['generated_text']}\n")
    return response[0]['generated_text']

In [4]:
dialog = [{"role": "user", "content": "what is the recipe of mayonnaise?"}]
prompt = format_messages(dialog)
payload = {"inputs": prompt, "parameters": {"max_new_tokens": 256, "top_p": 0.9, "temperature": 0.6}}
response = query_endpoint(payload)
print_messages(prompt, response)

> Input
<s>[INST] what is the recipe of mayonnaise? [/INST] 

> Output
Mayonnaise is a thick and creamy condiment made from a mixture of egg yolks, oil, vinegar or lemon juice, and seasonings. Here is a basic recipe for homemade mayonnaise:

Ingredients:

* 3 large egg yolks
* 1/2 cup (120 ml) neutral-tasting oil, such as canola or grapeseed
* 1 tablespoon (15 ml) vinegar or lemon juice
* 1/2 teaspoon salt
* 1/4 teaspoon black pepper

Instructions:

1. In a medium-sized bowl, whisk together the egg yolks and salt until they become light in color and thick.
2. Slowly pour the oil into the bowl while continuously whisking the mixture. You can start with a thin stream and gradually increase the flow of oil as the mixture thickens.
3. Once you have added all the oil, continue whisking the mixture for a few seconds to ensure it is smooth and emulsified.
4.



In [5]:
dialog = [
    {"role": "user", "content": "I am going to Paris, what should I see?"},
    {
        "role": "assistant",
        "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
    },
    {"role": "user", "content": "What is so great about #1?"},
]
prompt = format_messages(dialog)
payload = {"inputs": prompt, "parameters": {"max_new_tokens": 256, "top_p": 0.9, "temperature": 0.6}}
response = query_endpoint(payload)
print_messages(prompt, response)

> Input
<s>[INST] I am going to Paris, what should I see? [/INST] Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.</s><s>[INST] What is so great about #1? [/INST] 

> Output
There are many reaso

In [6]:
dialog = [
    {"role": "system", "content": "Always answer with Haiku"},
    {"role": "user", "content": "I am going to Paris, what should I see?"},
]
prompt = format_messages(dialog)
payload = {"inputs": prompt, "parameters": {"max_new_tokens": 256, "top_p": 0.9, "temperature": 0.6}}
response = query_endpoint(payload)
print_messages(prompt, response)

> Input
<s>[INST] <<SYS>>
Always answer with Haiku
<</SYS>>

I am going to Paris, what should I see? [/INST] 

> Output
Eiffel Tower high
River Seine's gentle flow
Art, love, and bread



In [7]:
dialog = [
    {"role": "system", "content": "Always answer with emojis"},
    {"role": "user", "content": "How to go from Beijing to NY?"},
]
prompt = format_messages(dialog)
payload = {"inputs": prompt, "parameters": {"max_new_tokens": 256, "top_p": 0.9, "temperature": 0.6}}
response = query_endpoint(payload)
print_messages(prompt, response)

> Input
<s>[INST] <<SYS>>
Always answer with emojis
<</SYS>>

How to go from Beijing to NY? [/INST] 

> Output
Here are the steps to go from Beijing to New York City:

🚀 Fly from Beijing Capital International Airport (PEK) to John F. Kennedy International Airport (JFK) or LaGuardia Airport (LGA) in New York City. The flight typically takes around 12-14 hours.

👉 You can fly with airlines such as Air China, China Eastern, or Delta.

🛣️ Take a taxi or ride-sharing service from the airport to your destination in New York City.

🚗 Alternatively, you can take a train or bus from Beijing to New York City, but this option is not as popular.

💭 Keep in mind that there may be travel restrictions or requirements due to the COVID-19 pandemic, so be sure to check the latest travel guidelines before planning your trip.



In [8]:
import os
import json
import gzip
import ast
import pandas as pd
from urllib.request import urlopen
import numpy as np
import re

In [15]:
df = pd.read_csv('/root/DemoNotebooks/hallucinations/Llama-customize/training/data/test.csv')

In [ ]:
### load the meta data

data = []
with gzip.open('meta_AMAZON_FASHION.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

In [ ]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

print(len(df))

In [ ]:
# Filter rows where column 'A' is not null
filtered_df = df[df['description'].notnull()]

In [ ]:
filtered_df = filtered_df[['title', 'brand', 'feature', 'description', 'price']]

In [ ]:
filtered_df = filtered_df.dropna(how='any')

In [ ]:
filtered_df

In [ ]:
#sample_df = df.head(20)

In [ ]:
#sample_df

In [16]:
def create_description_prompts(df):
    prompts = []
    for index, row in df.iterrows():
        # Check and convert the feature field
        feature = row['feature']
        if isinstance(feature, str):
            try:
                # Convert string representation of list to actual list
                feature_list = ast.literal_eval(feature)
            except (ValueError, SyntaxError):
                # If the string is not a list representation, make it a single-element list
                feature_list = [feature]
        elif isinstance(feature, list):
            feature_list = feature
        else:
            # If feature is neither a string nor a list, convert it to a string and make it a single-element list
            feature_list = [str(feature)]

        # Join the list elements into a string
        feature_list_str = '\n  - '.join(feature_list)
        
        prompt = (
            f"Title: {row['title']}\n"
            f"Brand: {row['brand']}\n"
            f"Price: ${row['price']}\n"
            f"Key Features: \n  - {feature_list_str}\n\n"
        )
        prompts.append(prompt)
    return prompts

In [17]:
# Initialize the DataFrame with 20 columns for descriptions
column_names = ['description_' + str(i) for i in range(1, 21)]
progress_df = pd.DataFrame(columns=column_names)
start_index = 0

# Generate prompts for each product in the part
prompts = create_description_prompts(df)

for index, prompt in enumerate(prompts, start=start_index):
    descriptions = []
    print('Starting generation for row: ' + str(index) + ' and prompt: ' + str(prompt))
    for sample in range(20):  # Loop to generate 20 samples per prompt
        try:
            dialog = [
                {"role": "system", "content": "Craft a vibrant and engaging product description of the item whose metadata is given. Transform these features into an alluring narrative that emphasizes the product's unique qualities. Highlight the practicality, quality, and any unique selling points, making it irresistible to potential buyers. Please provide only the product description in your response, without any additional commentary or explanation."},
                {"role": "user", "content": "Metadata of item:\n" + prompt},
            ]
            formatted_prompt = format_messages(dialog)
            payload = {"inputs": formatted_prompt, "parameters": {"do_sample": True, "max_new_tokens": 512, "top_p": 0.9, "temperature": 1.0, "return_full_text": True}}
            response = query_endpoint(payload)
            description = response[0]['generated_text']
        except Exception as e:
            description = None

        descriptions.append(description)

    # Create a new row with the 20 descriptions
    new_row = {column_names[i]: descriptions[i] for i in range(20)}

    # Add the new row to the DataFrame
    print('Completed iteration: ' + str(index))
    progress_df = progress_df.append(new_row, ignore_index=True)

Starting generation for row: 0 and prompt: Title: X. L. Carbon Fiber Money Clip, made in the USA
Brand: Roar Carbon
Price: $$14.99 
Key Features: 
  - Real Carbon Fiber
  - Made in USA
  - 5 year warranty against delamination
  - Holds Cash and Cards
  - You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused condition.
  - Read the full returns policy
  - Go to Your Orders to start the return
  - Print the return shipping label
  - Ship it!
  - Package Dimensions:
                    
2.5 x 2.2 x 0.6 inches
  - Shipping Weight:
                    
0.32 ounces (View shipping rates and policies)


Completed iteration: 0
Starting generation for row: 1 and prompt: Title: Buxton Heiress Pik-Me-Up Framed Case
Brand: Buxton
Price: $$16.95 
Key Features: 
  - Leather
  - Imported
  - synthetic lining
  - Flap closure
  - Authentic Crunch Leather
  - Rich Floral Embossed Logo
  - Goldtone Hardware
  - Large Snap Pocket
  

/tmp/ipykernel_11136/843936704.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


Completed iteration: 1
Starting generation for row: 2 and prompt: Title: Art Nouveau Sterling Silver Ornate Repousse Heart Bookmark - Book Lovers Gift
Brand: Silver Insanity
Price: $$44.66 
Key Features: 
  - 2&5/8" High and 3/4" Wide
  - Weight is Approx. 9 Grams
  - Marked 925
  - Vintage Inspired Style
  - Heart Bookmark
  - You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused condition.
  - Read the full returns policy
  - Go to Your Orders to start the return
  - Print the return shipping label
  - Ship it!
  - Package Dimensions:
                    
7.5 x 3.4 x 0.7 inches
  - Shipping Weight:
                    
0.8 ounces (View shipping rates and policies)




/tmp/ipykernel_11136/843936704.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


Completed iteration: 2
Starting generation for row: 3 and prompt: Title: Dream PJ's Blue - Large - Part #: 25BLG
Brand: Ethical/Spot
Price: $$15.99 
Key Features: 
  - Product Dimensions:
                    
8.4 x 3.8 x 0.4 inches
  - Shipping Weight:
                    
5 ounces (View shipping rates and policies)




...

/tmp/ipykernel_11136/843936704.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


Completed iteration: 15
Starting generation for row: 16 and prompt: Title: CliC Adjustable Front Connect Universal Sport Goggle, Black Frame
Brand: CliC
Price: $$39.95 - $49.95
Key Features: 
  - Package Dimensions:
                    
10.2 x 5.9 x 2.6 inches
  - Shipping Weight:
                    
4 ounces (View shipping rates and policies)




/tmp/ipykernel_11136/843936704.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


Completed iteration: 16


/tmp/ipykernel_11136/843936704.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


In [18]:
progress_df

description_1  \
0   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
1   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
2   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
3   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
4   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
5   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
6   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
7   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
8   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
9   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
10  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
11  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
12  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
13  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
14  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
15  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
16  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   

                                        description_2  \
0   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
1   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
2   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
3   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
4   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
5   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
6   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
7   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
8   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
9   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
10  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
11  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
12  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
13  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
14  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
15  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
16  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   

                                        description_3  \
0   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
1   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
2   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
3   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
4   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
5   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
6   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
7   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
8   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
9   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
10  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
11  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
12  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
13  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
14  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
15  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
16  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   

                                        description_4  \
0   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
1   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
2   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
3   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
4   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
5   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
6   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
7   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
8   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
9   <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
10  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
11  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
12  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
13  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
14  <s>[INST] <<SYS>>\nCraft a vibrant and engagin...   
15  <s>[INST] <<SYS>>\nCraft a vibrant and enga

In [ ]:
# Concatenate the DataFrames horizontally
#sample_df = pd.concat([sample_df, progress_df], axis=1)

In [ ]:
#sample_df

In [ ]:
#sample_df.to_csv('test.csv', index=False)

In [ ]:
# Fine-tuning exercise

In [ ]:
#finetune_df = filtered_df.iloc[20:]

In [ ]:
# Generate prompts for each product in the fine tuned df
filtered_prompts = create_description_prompts(filtered_df)

In [ ]:
if len(filtered_prompts) != len(filtered_df):
    raise ValueError("The number of prompts does not match the number of rows in the DataFrame.")

# Create a new DataFrame from the prompts list
prompts_df = pd.DataFrame({'prompt': filtered_prompts})

# Concatenate the new DataFrame with the original DataFrame
filtered_df['prompt'] = prompts_df['prompt']

In [ ]:
filtered_df

In [ ]:
# Read the CSV file into a DataFrame
csv_df = pd.read_csv('generated_descriptions.csv')

# Ensure that both DataFrames have the same number of rows
if len(csv_df) != len(filtered_df):
    raise ValueError("The number of rows in the CSV does not match the number of rows in the source DataFrame.")

# Assuming the CSV file has a single column or you're interested in a specific column, for example 'new_column'
filtered_df['generated_description'] = csv_df['generated_description']

In [ ]:
filtered_df.head(20)

In [ ]:
filtered_df

In [ ]:
filtered_df.to_csv('data_finetuned.csv', index=True)

In [ ]:
import pandas as pd
import json

# Define the fixed instruction
instruction = "Craft a vibrant and engaging product description of the item whose metadata is given. Transform these features into an alluring narrative that emphasizes the product's unique qualities. Highlight the practicality, quality, and any unique selling points, making it irresistible to potential buyers."

# Create a new column for the JSON structure
filtered_df['json_structure'] = filtered_df.apply(lambda row: {
    "prompt": f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{row['prompt']}",
    "completion": row['description']
}, axis=1)

'''# Convert the new column to JSON
json_data = filtered_df['json_structure'].to_json(orient='records', lines=True)

# Write to a JSON file
with open('output.json', 'w') as file:
    file.write(json_data)

# Output the JSON data (optional)
print(json_data)'''

In [ ]:
# Using a regular expression to match the end of the key features list and extract the product description
def extract_description_after_features(text):
    # Match everything after the key features list, assuming it ends with the closing bracket of the list
    match = re.search(r'Key Features:.*?\n\n', text, re.DOTALL)
    if match:
        # Extract the text after the key features list
        return text[match.end():].strip()
    else:
        # If no match is found, return the original text indicating there was no key features list
        return text

# Apply the extraction function to every description in the dataframe
for col in df_new.columns:
    if "description" in col:
        df_new[col] = df_new[col].apply(extract_description_after_features)

# Save the updated dataframe to a new CSV file
output_file_path_description_extracted = '/mnt/data/description_extracted.csv'
df_new.to_csv(output_file_path_description_extracted, index=False)

output_file_path_description_extracted

In [19]:
progress_df.to_csv('/root/DemoNotebooks/hallucinations/Llama-customize/training/data/new_test.csv', index=True)

In [66]:
final_df = pd.read_csv('/root/DemoNotebooks/hallucinations/Llama-customize/training/data/description_extracted.csv')

In [67]:
# Initialize the DataFrame with 1 column for descriptions
column_names = ['description']
progress_df = pd.DataFrame(columns=column_names)
start_index = 0

for index, prompt in enumerate(prompts, start=start_index):
    print('Starting generation for row: ' + str(index) + ' and prompt: ' + str(prompt))
    try:
        dialog = [
            {"role": "system", "content": "Craft a vibrant and engaging product description of the item whose metadata is given. Transform these features into an alluring narrative that emphasizes the product's unique qualities. Highlight the practicality, quality, and any unique selling points, making it irresistible to potential buyers. Please provide only the product description in your response, without any additional commentary or explanation."},
            {"role": "user", "content": "Metadata of item:\n" + prompt},
        ]
        formatted_prompt = format_messages(dialog)
        payload = {"inputs": formatted_prompt, "parameters": {"do_sample": True, "max_new_tokens": 512, "top_p": 0.9, "temperature": 1.0, "return_full_text": True}}
        response = query_endpoint(payload)
        description = response[0]['generated_text']
    except Exception as e:
        description = None

    # Create a new row with the single description
    new_row = {column_names[0]: description}

    # Add the new row to the DataFrame
    print('Completed iteration: ' + str(index))
    final_df = final_df.append(new_row, ignore_index=True)


Starting generation for row: 0 and prompt: Title: X. L. Carbon Fiber Money Clip, made in the USA
Brand: Roar Carbon
Price: $$14.99 
Key Features: 
  - Real Carbon Fiber
  - Made in USA
  - 5 year warranty against delamination
  - Holds Cash and Cards
  - You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused condition.
  - Read the full returns policy
  - Go to Your Orders to start the return
  - Print the return shipping label
  - Ship it!
  - Package Dimensions:
                    
2.5 x 2.2 x 0.6 inches
  - Shipping Weight:
                    
0.32 ounces (View shipping rates and policies)


Completed iteration: 0
Starting generation for row: 1 and prompt: Title: Buxton Heiress Pik-Me-Up Framed Case
Brand: Buxton
Price: $$16.95 
Key Features: 
  - Leather
  - Imported
  - synthetic lining
  - Flap closure
  - Authentic Crunch Leather
  - Rich Floral Embossed Logo
  - Goldtone Hardware
  - Large Snap Pocket
  

/tmp/ipykernel_11136/220073429.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(new_row, ignore_index=True)


Completed iteration: 1
Starting generation for row: 2 and prompt: Title: Art Nouveau Sterling Silver Ornate Repousse Heart Bookmark - Book Lovers Gift
Brand: Silver Insanity
Price: $$44.66 
Key Features: 
  - 2&5/8" High and 3/4" Wide
  - Weight is Approx. 9 Grams
  - Marked 925
  - Vintage Inspired Style
  - Heart Bookmark
  - You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused condition.
  - Read the full returns policy
  - Go to Your Orders to start the return
  - Print the return shipping label
  - Ship it!
  - Package Dimensions:
                    
7.5 x 3.4 x 0.7 inches
  - Shipping Weight:
                    
0.8 ounces (View shipping rates and policies)




/tmp/ipykernel_11136/220073429.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(new_row, ignore_index=True)


Completed iteration: 2
Starting generation for row: 3 and prompt: Title: Dream PJ's Blue - Large - Part #: 25BLG
Brand: Ethical/Spot
Price: $$15.99 
Key Features: 
  - Product Dimensions:
                    
8.4 x 3.8 x 0.4 inches
  - Shipping Weight:
                    
5 ounces (View shipping rates and policies)




...

/tmp/ipykernel_11136/220073429.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(new_row, ignore_index=True)


Completed iteration: 15
Starting generation for row: 16 and prompt: Title: CliC Adjustable Front Connect Universal Sport Goggle, Black Frame
Brand: CliC
Price: $$39.95 - $49.95
Key Features: 
  - Package Dimensions:
                    
10.2 x 5.9 x 2.6 inches
  - Shipping Weight:
                    
4 ounces (View shipping rates and policies)




/tmp/ipykernel_11136/220073429.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(new_row, ignore_index=True)


Completed iteration: 16


/tmp/ipykernel_11136/220073429.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(new_row, ignore_index=True)


In [68]:
final_df

Unnamed: 0                                      description_1  \
0          0.0  Introducing the sleek and sophisticated X. L. ...   
1          1.0  Introducing the Buxton Heiress Pik-Me-Up Frame...   
2          2.0  Measuring 2&5/8" high and 3/4" wide, this hear...   
3          3.0  Dive into a dreamy slumber with our ethically-...   
4          4.0  Transform into a feline fashionista with the S...   
5          5.0  Elevate your everyday carry with the stylish a...   
6          6.0  Indulge in the beauty and elegance of this exq...   
7          7.0  Indulge in the coziest and softest sleep of yo...   
8          8.0  Experience the luxury of true-to-size sizing, ...   
9          9.0  Gem Avenue's 925 Sterling Silver Cubic Zirconi...   
10        10.0  Elevate your jewelry collection with the stunn...   
11        11.0  But that's not all - with our hassle-free retu...   
12        12.0  Experience the grit and grace of the cowboy wa...   
13        13.0  The Schaefer Outfitters Legend Denim Jacket fe...   
14        14.0  With multiple pockets and compartments, you'll...   
15        15.0  The Stockman Vest features a sleek and stylish...   
16        16.0  Experience the freedom to wear your goggles yo...   
17         NaN                                                NaN   
18         NaN                                                NaN   
19         NaN                                                NaN   
20         NaN                                                NaN   
21         NaN                                                NaN   
22         NaN                                                NaN   
23         NaN                                                NaN   
24         NaN                                                NaN   
25         NaN                                                NaN   
26         NaN                                                NaN   
27         NaN                                                NaN   
28         NaN                                                NaN   
29         NaN                                                NaN   
30         NaN                                                NaN   
31         NaN                                                NaN   
32         NaN                                                NaN   
33         NaN                                                NaN   

                                        description_2  \
0   Experience the luxury of a high-end money clip...   
1   The flap closure and goldtone hardware add a t...   
2   Measuring 2&5/8" high and 3/4" wide, this hear...   
3   Indulge in the coziest sleep of your life with...   
4   🎉 Get ready to purr-fect your Halloween look w...   
5   Elevate your everyday carry with the stylish a...   
6   💖 Unleash Your Love for Books with This Exquis...   
7   Immerse yourself in the ultimate comfort of ou...   
8   Step into the spotlight with the Pleaser Women...   
9   Introducing the 925 Sterling Silver Cubic Zirc...   
10  Indulge in the sparkling elegance of this 925 ...   
11  With our risk-free return policy, you can try ...   
12  Spin a Yarn with Our Vintage Cowboy Ropes! 🎁\n...   
13  With its timeless design and versatile fit, th...   
14  With a durable water-resistant nylon construct...   
15  Introducing the Stockman Vest from Schaefer Ou...   
16  At just 4 ounces, this goggle is lightweight a...   
17                                                NaN   
18                                                NaN   
19                                                NaN   
20                                                NaN   
21                                                NaN   
22                                                NaN   
23                                                NaN   
24                                                NaN   
25                                                NaN   
26                                               

In [69]:
final_df.to_csv('/root/DemoNotebooks/hallucinations/Llama-customize/training/data/updated_test.csv', index=True)